In [89]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import os, sys, glob
import numpy as np
from tqdm import tqdm
import torch
import smplx
from sklearn.neighbors import NearestNeighbors
from scipy.spatial.transform import Rotation as R
from scipy.ndimage import gaussian_filter1d
import json
import csv
import pdb
import pickle

In [90]:
def get_body_model(type, gender, batch_size,device='cpu'):
    '''
    type: smpl, smplx smplh and others. Refer to smplx tutorial
    gender: male, female, neutral
    batch_size: an positive integar
    '''
    body_model_path = '/home/yuxinyao/body_models/'
    body_model = smplx.create(body_model_path, model_type=type,
                                    gender=gender, ext='npz',
                                    num_pca_comps=12,
                                    create_global_orient=True,
                                    create_body_pose=True,
                                    create_betas=True,
                                    create_left_hand_pose=True,
                                    create_right_hand_pose=True,
                                    create_expression=True,
                                    create_jaw_pose=True,
                                    create_leye_pose=True,
                                    create_reye_pose=True,
                                    create_transl=True,
                                    batch_size=batch_size
                                    )
    if device == 'cuda':
        return body_model.cuda()
    else:
        return body_model
        

In [91]:
def get_new_coordinate(bm_one, transl,global_orient,pose,betas):
    temp_bodyconfig = {}
    temp_bodyconfig['transl'] = torch.FloatTensor(transl).cuda()
    temp_bodyconfig['global_orient'] = torch.FloatTensor(global_orient).cuda()
    temp_bodyconfig['body_pose'] = torch.FloatTensor(pose).cuda()
    temp_bodyconfig['betas'] = torch.FloatTensor(betas).cuda()

    smplxout = bm_one(**temp_bodyconfig)
    joints = smplxout.joints.squeeze().detach().cpu().numpy()
    x_axis = joints[2,:] - joints[1,:]
    x_axis[-1] = 0
    x_axis = x_axis / np.linalg.norm(x_axis)
    z_axis = np.array([0,0,-1])
    y_axis = np.cross(z_axis, x_axis)
    y_axis = y_axis/np.linalg.norm(y_axis)
    global_ori_new = np.stack([x_axis, y_axis, z_axis], axis=1)
    transl_new = joints[:1,:] # put the local origin to pelvis


    transf_rotmat = global_ori_new
    transf_transl = transl_new

    return transf_rotmat,transf_transl

In [92]:
egobody_dataset_path = '/home/yuxinyao/datasets/egobody/smplx_camera_wearer'
with open('/home/yuxinyao/body_models/Mosh_related/CMU.json') as f:
        marker_cmu_41 = list(json.load(f)['markersets'][0]['indices'].values())

with open('/home/yuxinyao/body_models/Mosh_related/SSM2.json') as f:
        marker_ssm_67 = list(json.load(f)['markersets'][0]['indices'].values())


folder_seqs = glob.glob(os.path.join(egobody_dataset_path,"recording*"))

len_subseq = 10
bm_one_male = get_body_model('smplx','male',1,device = 'cuda')
bm_one_female = get_body_model('smplx','female',1,device='cuda')
bm_batch_male = get_body_model('smplx','male',len_subseq,device='cuda')
bm_batch_female = get_body_model('smplx','female',len_subseq,device='cuda')

In [88]:
# for record in folder_seqs:
#     seqs = sorted(glob.glob(os.path.join(record, '*/*/*/*.pkl')))
#     for i in range(0, len(seqs),len_subseq):
#         first_frame_path = seqs[i]
#         with open(first_frame_path,'rb') as f:
#             first_frame = dict(pickle.load(f))
#         transl_seq = np.zeros((len_subseq,3))
#         pose_seq = np.zeros((len_subseq,63))
#         betas_seq = np.zeros((len_subseq,10))
#         betas = first_frame['betas']
#         global_orient_seq = np.zeros((len_subseq,3))
#         bodymodel_batch = bm_batch_male if str(first_frame['gender']) =='male' else bm_batch_female
#         bodymodel_one = bm_one_male if str(first_frame['gender']) =='male' else bm_one_female

#         for j in range(0,len_subseq):
#             seq = seqs[j]
#             with open(seq,'rb') as f:
#                 data = dict(pickle.load(f))
#             transl = data['transl']
#             pose = data['body_pose']
#             global_orient = data['global_orient']
#             betas = data['betas'][:10]
#             transl_seq[j,:] = transl
#             pose_seq[j,:]= pose
#             global_orient_seq[j,:] = global_orient
#             betas_seq[j,:] = betas

#         data_out = {}
#         outfolder = "/home/yuxinyao/datasets/egobody/canicalized-camera-wearer/" + str(record[49:])
#         if not os.path.exists(outfolder):
#             os.makedirs(outfolder)
#         outfilename = os.path.join(outfolder, 'subseq_{:05d}1.pkl'.format(i))


        
#         # transf_rotmat, transf_transl = get_new_coordinate(bodymodel_one, first_frame['transl'], first_frame['global_orient'],first_frame['body_pose'],first_frame['betas'])
#         # get_new_coordinate
#         temp_bodyconfig = {}
#         temp_bodyconfig['transl'] = torch.FloatTensor(first_frame['transl']).cuda()
#         temp_bodyconfig['global_orient'] = torch.FloatTensor(first_frame['global_orient']).cuda()
#         temp_bodyconfig['body_pose'] = torch.FloatTensor(first_frame['body_pose']).cuda()
#         temp_bodyconfig['betas'] = torch.FloatTensor(first_frame['betas']).cuda()

#         smplxout = bodymodel_one(**temp_bodyconfig)
#         joints = smplxout.joints.squeeze().detach().cpu().numpy()
#         x_axis = joints[2,:] - joints[1,:]
#         x_axis[-1] = 0
#         x_axis = x_axis / np.linalg.norm(x_axis)
#         z_axis = np.array([0,0,-1])
#         y_axis = np.cross(z_axis, x_axis)
#         y_axis = y_axis/np.linalg.norm(y_axis)
#         global_ori_new = np.stack([x_axis, y_axis, z_axis], axis=1)
#         # transl_new = joints[:1,:] # put the local origin to pelvis


#         transf_rotmat = global_ori_new
#         # transf_transl = transl_new


# #############################################################################################################################  


#         temp_bodyconfig_seq = {}
#         temp_bodyconfig_seq['transl'] = torch.FloatTensor(transl_seq).cuda()
#         temp_bodyconfig_seq['global_orient'] = torch.FloatTensor(global_orient_seq).cuda()
#         temp_bodyconfig_seq['body_pose'] = torch.FloatTensor(pose_seq).cuda()
#         temp_bodyconfig_seq['betas'] = torch.FloatTensor(betas_seq).cuda()

#         smplx_out = bodymodel_batch(return_verts=True, **temp_bodyconfig_seq)
#         delta_T = smplx_out.joints[:,0,:] # we output all pelvis locations
#         delta_T = delta_T.detach().cpu().numpy() #[t, 3]

#         global_ori_new = np.einsum('ij,tjk->tik', transf_rotmat.T, R.from_rotvec(global_orient_seq).as_matrix())
#         # print(global_orient_seq.shape)

#         # temp_bodyconfig_seq['global_orient'] = torch.FloatTensor(R.from_matrix(global_ori_new).as_rotvec()).cuda()
        
#         # smplx_out = bodymodel_batch(return_verts=True, **temp_bodyconfig_seq)
#         # delta_T = smplx_out.joints[:,0,:] # we output all pelvis locations
#         # delta_T = delta_T.detach().cpu().numpy() #[t, 3]

#         global_ori_new = np.einsum('ij,tjk->tik', R.from_rotvec([-np.pi/2,0,0]).as_matrix(),R.from_rotvec(global_orient_seq).as_matrix())
#         global_orient_save = R.from_matrix(global_ori_new).as_rotvec()

#         # temp_bodyconfig_seq['global_orient'] = torch.FloatTensor(global_orient_save).cuda()
        
#         # smplx_out = bodymodel_batch(return_verts=True, **temp_bodyconfig_seq)
#         # delta_T = smplx_out.joints[:,0,:] # we output all pelvis locations
#         # delta_T = delta_T.detach().cpu().numpy() #[t, 3]

# #############################################################################################################################
#         # print(global_orient_save[:1,:].shape)
#         temp_bodyconfig['global_orient'] = torch.FloatTensor(global_orient_save[:1,:]).cuda()
#         smplxout = bodymodel_one(**temp_bodyconfig)
#         joints = smplxout.joints.squeeze().detach().cpu().numpy()
#         transf_transl = joints[:1,:]
# #############################################################################################################################
#         # print(transl_seq)
#         transl_save = np.einsum('ij,tj->ti',R.from_rotvec([-np.pi/2,0,0]).as_matrix(),transl_seq)
#         # print(transl_save)
#         # transl_save = np.einsum('ij,tj->ti', transf_rotmat.T, transl_seq+delta_T-transf_transl)-delta_T
#         transl_save = np.einsum('ij,tj->ti', transf_rotmat.T, transl_save+delta_T-transf_transl)-delta_T
#         # transl_save = np.einsum('ij,tj->ti', transf_rotmat.T, transl_seq+delta_T-transf_transl)-delta_T
#         # transl_save = np.einsum('ij,tj->ti', transf_rotmat.T, transl_seq) - delta_T
#         # print(transl_save)
#         data_out['trans'] = transl_save
#         data_out['body_pose'] = pose_seq
#         data_out['betas'] = betas_seq
#         data_out['gender'] = data['gender']
#         data_out['global_orient'] = global_orient_save
#         data_out['transf_rotmat'] = transf_rotmat
#         data_out['transf_transl'] = transf_transl
#         # data_out['mocap_framerate'] = 30

#         # print(global_orient_save.shape)
#         body_param = {}
#         body_param['transl'] = torch.FloatTensor(transl_save).cuda()
#         body_param['global_orient'] = torch.FloatTensor(global_orient_save).cuda()
#         body_param['betas'] = torch.FloatTensor(betas_seq).cuda()
#         body_param['body_pose'] = torch.FloatTensor(pose_seq).cuda()
#         smplxout = bodymodel_batch(return_verts=True, **body_param)
#         joints = smplxout.joints[:,:22,:].detach().squeeze().cpu().numpy()
#         print("joints:"+str(joints[0,0,:]))
#         markers_41 = smplxout.vertices[:,marker_cmu_41,:].detach().squeeze().cpu().numpy()
#         markers_67 = smplxout.vertices[:,marker_ssm_67,:].detach().squeeze().cpu().numpy()
#         data_out['joints'] = joints
#         data_out['marker_cmu_41'] = markers_41
#         data_out['marker_ssm2_67'] = markers_67
#         with open(outfilename, 'wb') as f:
#             pickle.dump(data_out,f)
#         print(outfilename)
        
#         break
#     break




joints:[-0.19068965 -2.8048675  -2.2560668 ]
/home/yuxinyao/datasets/egobody/canicalized-camera-wearer/er/recording_20210910_S06_S05_01/subseq_000001.pkl


In [111]:
for record in folder_seqs:
    seqs = sorted(glob.glob(os.path.join(record, '*/*/*/*.pkl')))
    for i in range(0, len(seqs),len_subseq):
        first_frame_path = seqs[i]
        with open(first_frame_path,'rb') as f:
            first_frame = dict(pickle.load(f))
        transl_seq = np.zeros((len_subseq,3))
        pose_seq = np.zeros((len_subseq,63))
        betas_seq = np.zeros((len_subseq,10))
        betas = first_frame['betas']
        global_orient_seq = np.zeros((len_subseq,3))
        bodymodel_batch = bm_batch_male if str(first_frame['gender']) =='male' else bm_batch_female
        bodymodel_one = bm_one_male if str(first_frame['gender']) =='male' else bm_one_female

        for j in range(0,len_subseq):
            seq = seqs[j]
            with open(seq,'rb') as f:
                data = dict(pickle.load(f))
            transl = data['transl']
            pose = data['body_pose']
            global_orient = data['global_orient']
            betas = data['betas'][:10]
            transl_seq[j,:] = transl
            pose_seq[j,:]= pose
            global_orient_seq[j,:] = global_orient
            betas_seq[j,:] = betas

        data_out = {}
        outfolder = "/home/yuxinyao/datasets/egobody/canicalized-camera-wearer/" + str(record[49:])
        if not os.path.exists(outfolder):
            os.makedirs(outfolder)
        outfilename = os.path.join(outfolder, 'subseq_{:05d}1test.pkl'.format(i))

# rotate to yan's coordinate 
        temp_bodyconfig = {}
        temp_bodyconfig['transl'] = torch.FloatTensor(first_frame['transl']).cuda()
        temp_bodyconfig['global_orient'] = torch.FloatTensor(first_frame['global_orient']).cuda()
        temp_bodyconfig['body_pose'] = torch.FloatTensor(first_frame['body_pose']).cuda()
        temp_bodyconfig['betas'] = torch.FloatTensor(first_frame['betas']).cuda()

        smplxout = bodymodel_one(**temp_bodyconfig)
        joints = smplxout.joints.squeeze().detach().cpu().numpy()
        # x_axis = joints[2,:] - joints[1,:]
        # x_axis[-1] = 0
        # x_axis = x_axis / np.linalg.norm(x_axis)
        # z_axis = np.array([0,0,-1])
        # y_axis = np.cross(z_axis, x_axis)
        # y_axis = y_axis/np.linalg.norm(y_axis)
        # global_ori_new = np.stack([x_axis, y_axis, z_axis], axis=1)
        global_ori_rotate = R.from_rotvec([np.pi/2,0,0]).as_matrix()
        transl_new = joints[:1,:] # put the local origin to pelvis


        transf_rotmat1 = global_ori_rotate
        transf_transl1 = transl_new

        temp_bodyconfig_seq = {}
        temp_bodyconfig_seq['transl'] = torch.FloatTensor(transl_seq).cuda()
        temp_bodyconfig_seq['global_orient'] = torch.FloatTensor(global_orient_seq).cuda()
        temp_bodyconfig_seq['body_pose'] = torch.FloatTensor(pose_seq).cuda()
        temp_bodyconfig_seq['betas'] = torch.FloatTensor(betas_seq).cuda()

        smplx_out = bodymodel_batch(return_verts=True, **temp_bodyconfig_seq)
        delta_T = smplx_out.joints[:,0,:] # we output all pelvis locations
        delta_T = delta_T.detach().cpu().numpy() #[t, 3]

        global_ori_1 = R.from_matrix(np.einsum('ij,tjk->tik', transf_rotmat1.T, R.from_rotvec(global_orient_seq).as_matrix())).as_rotvec()
        transl1 = np.einsum('ij,tj->ti', transf_rotmat1.T, transl_seq+delta_T-transl_new)-delta_T

#calibrate body to center 
        print(type(global_ori_1))
        temp_bodyconfig['global_orient'] = torch.FloatTensor(global_ori_1[:1,:]).cuda()
        temp_bodyconfig['transl'] = torch.FloatTensor(transl1[:1,:]).cuda()
        
        smplxout = bodymodel_one(**temp_bodyconfig)

        joints = smplxout.joints.squeeze().detach().cpu().numpy()
        x_axis = joints[2,:] - joints[1,:]
        x_axis[-1] = 0
        x_axis = x_axis / np.linalg.norm(x_axis)
        z_axis = np.array([0,0,1])
        y_axis = np.cross(z_axis, x_axis)
        y_axis = y_axis/np.linalg.norm(y_axis)
        global_ori_new = np.stack([x_axis, y_axis, z_axis], axis=1)
        transl_new = joints[:1,:] # put the local origin to pelvis

        transf_rotmat = global_ori_new
        transf_transl = transl_new

        temp_bodyconfig_seq['transl'] = torch.FloatTensor(transl1).cuda()
        temp_bodyconfig_seq['global_orient'] = torch.FloatTensor(global_ori_1).cuda()

        smplx_out = bodymodel_batch(return_verts=True, **temp_bodyconfig_seq)
        delta_T = smplx_out.joints[:,0,:] # we output all pelvis locations
        delta_T = delta_T.detach().cpu().numpy() #[t, 3]

        global_ori_save = R.from_matrix(np.einsum('ij,tjk->tik', transf_rotmat.T, R.from_rotvec(global_ori_1).as_matrix())).as_rotvec()
        transl_save = np.einsum('ij,tj->ti', transf_rotmat.T, transl1+delta_T-transl_new)-delta_T


# #############################################################################################################################
#         temp_bodyconfig['global_orient'] = torch.FloatTensor(global_orient_save[:1,:]).cuda()
#         smplxout = bodymodel_one(**temp_bodyconfig)
#         joints = smplxout.joints.squeeze().detach().cpu().numpy()
#         transf_transl = joints[:1,:]
# #############################################################################################################################
#         transl_save = np.einsum('ij,tj->ti',R.from_rotvec([-np.pi/2,0,0]).as_matrix(),transl_seq)
#         transl_save = np.einsum('ij,tj->ti', transf_rotmat.T, transl_save+delta_T-transf_transl)-delta_T
        data_out['trans'] = transl_save
        data_out['body_pose'] = pose_seq
        data_out['betas'] = betas_seq
        data_out['gender'] = data['gender']
        data_out['global_orient'] = global_ori_save
        # data_out['transf_rotmat'] = transf_rotmat
        # data_out['transf_transl'] = transf_transl
#         # data_out['mocap_framerate'] = 30

#         # print(global_orient_save.shape)
        body_param = {}
        body_param['transl'] = torch.FloatTensor(transl_save).cuda()
        body_param['global_orient'] = torch.FloatTensor(global_ori_save).cuda()
        body_param['betas'] = torch.FloatTensor(betas_seq).cuda()
        body_param['body_pose'] = torch.FloatTensor(pose_seq).cuda()
        smplxout = bodymodel_batch(return_verts=True, **body_param)
        joints = smplxout.joints[:,:22,:].detach().squeeze().cpu().numpy()
        print("joints:"+str(joints[0,0,:]))
        markers_41 = smplxout.vertices[:,marker_cmu_41,:].detach().squeeze().cpu().numpy()
        markers_67 = smplxout.vertices[:,marker_ssm_67,:].detach().squeeze().cpu().numpy()
        data_out['joints'] = joints
        data_out['marker_cmu_41'] = markers_41
        data_out['marker_ssm2_67'] = markers_67
        with open(outfilename, 'wb') as f:
            pickle.dump(data_out,f)
        print(outfilename)
        
        break
    break


<class 'numpy.ndarray'>
joints:[ 8.5427964e-01 -1.4420073e-01 -2.3283064e-08]
/home/yuxinyao/datasets/egobody/canicalized-camera-wearer/er/recording_20210910_S06_S05_01/subseq_000001test.pkl


In [83]:
with open('/home/yuxinyao/datasets/egobody/canicalized-camera-wearer/er/recording_20210910_S06_S05_01/subseq_000001.pkl', 'rb') as f:
    read = dict(pickle.load(f))


In [84]:
bm = get_body_model('smplx', read['gender'], 10, device='cpu')
bparam = {}
bparam['trans'] = read['trans']
# print("transl: "+str(bparam['trans']))
bparam['global_orient'] = read['global_orient']
bparam['betas'] = read['betas']
bparam['body_pose'] = read['body_pose']
# bparam['left_hand_pose'] = data_bparams[:,69:81]
# bparam['right_hand_pose'] = data_bparams[:,81:]


## obtain body mesh sequences
for key in bparam:
    # if key == 'betas':
    #     bparam[key] = torch.tensor(bparam[key], dtype=torch.float32)
    # else:
    bparam[key] = torch.FloatTensor(bparam[key])
    print("bparam[{}] shape: {}".format(key, bparam[key].shape))
# print("bm.batchsize = "+str(bm.batch_size))
# print(bparam)
verts_seq = bm(return_verts=True, **bparam).vertices.detach().cpu().numpy() #[t,verts, 3]
jts = bm(return_verts=True, **bparam).joints[:,:22].detach().cpu().numpy() #[t,J, 3]

bparam[trans] shape: torch.Size([10, 3])
bparam[global_orient] shape: torch.Size([10, 3])
bparam[betas] shape: torch.Size([10, 10])
bparam[body_pose] shape: torch.Size([10, 63])


In [85]:
jts[:,0,:]

array([[ 0.00093888, -0.34094906,  0.01028696],
       [ 0.00093888, -0.34094906,  0.01028696],
       [ 0.00093888, -0.34094906,  0.01028696],
       [ 0.00093888, -0.34094906,  0.01028696],
       [ 0.00093888, -0.34094906,  0.01028696],
       [ 0.00093888, -0.34094906,  0.01028696],
       [ 0.00093888, -0.34094906,  0.01028696],
       [ 0.00093888, -0.34094906,  0.01028696],
       [ 0.00093888, -0.34094906,  0.01028696],
       [ 0.00093888, -0.34094906,  0.01028696]], dtype=float32)

In [77]:
jts[:,0,:]

array([[ 0.00093888, -0.34094906,  0.01028696],
       [ 0.00093888, -0.34094906,  0.01028696],
       [ 0.00093888, -0.34094906,  0.01028696],
       [ 0.00093888, -0.34094906,  0.01028696],
       [ 0.00093888, -0.34094906,  0.01028696],
       [ 0.00093888, -0.34094906,  0.01028696],
       [ 0.00093888, -0.34094906,  0.01028696],
       [ 0.00093888, -0.34094906,  0.01028696],
       [ 0.00093888, -0.34094906,  0.01028696],
       [ 0.00093888, -0.34094906,  0.01028696]], dtype=float32)

In [1]:
import pickle
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import os, sys, glob
import numpy as np
from tqdm import tqdm
import torch
import smplx
from sklearn.neighbors import NearestNeighbors
from scipy.spatial.transform import Rotation as R
from scipy.ndimage import gaussian_filter1d
import json
import csv
import pdb
import pickle

In [2]:
with open('/home/yuxinyao/datasets/egobody/canicalized-camera-wearer-grab-openpose/recording_20220312_S28_S29_04/subseq_02622.pkl', 'rb') as f:
    data = dict(pickle.load(f))

with open('/home/yuxinyao/datasets/egobody/canicalized-camera-wearer-grab-openpose-unchanged/recording_20220312_S28_S29_04/subseq_02622.pkl', 'rb') as f:
    originalf = dict(pickle.load(f))

In [3]:
testbp = {}
testbp['transl'] = data['trans']
testbp['global_orient'] = data['global_orient']
testbp['betas'] = data['betas']
testbp['body_pose'] = data['body_pose']
testbp['left_hand_pose'] = data['left_hand_pose']
testbp['right_hand_pose'] = data['right_hand_pose']

original = {}
original['transl'] = originalf['trans']
original['global_orient'] = originalf['global_orient']
original['betas'] = originalf['betas']
original['body_pose'] = originalf['body_pose']
original['left_hand_pose'] = originalf['left_hand_pose']
original['right_hand_pose'] = originalf['right_hand_pose']

In [16]:
testbp['transf_rotmat'] = data['transf_rotmat'] 
testbp['transf_transl'] = data['transf_transl'] 
testbp['delta_T'] = data['delta_T']
testbp['transf_rotate_xaxis'] = data['transf_rotate_xaxis']
testbp['transf_transl_xaxis'] = data['transf_transl_xaxis']
testbp['delta_T_rotate'] = data['delta_T_rotate']
transf_rotmat =testbp['transf_rotmat']
transf_transl =testbp['transf_transl']
delta_T=testbp['delta_T']
transf_rotate_xaxis=testbp['transf_rotate_xaxis']
transf_transl_xaxis=testbp['transf_transl_xaxis']
delta_T_rotate=testbp['delta_T_rotate']

In [5]:
original['transl']

array([[-1.40475595,  0.50640678,  2.51098347],
       [-1.40515757,  0.50646621,  2.51406598],
       [-1.40560806,  0.50657243,  2.51727271],
       [-1.40592551,  0.5067845 ,  2.52022505],
       [-1.40634954,  0.50716275,  2.52266359],
       [-1.40668905,  0.50763053,  2.52489638],
       [-1.40683055,  0.50832957,  2.52634668],
       [-1.40707588,  0.50922692,  2.52783298],
       [-1.4072057 ,  0.50985724,  2.52893209],
       [-1.40764618,  0.51034427,  2.53017306]])

In [6]:
testbp['transl']

array([[-0.00044794,  0.35737169, -0.01654189],
       [-0.00323977,  0.35600461, -0.01660131],
       [-0.00613363,  0.35455147, -0.01670753],
       [-0.00882906,  0.35330578, -0.0169196 ],
       [-0.01100361,  0.35212358, -0.01729785],
       [-0.01301029,  0.3510873 , -0.01776563],
       [-0.01433901,  0.35048906, -0.01846467],
       [-0.0156686 ,  0.34978092, -0.01936202],
       [-0.01666835,  0.34930615, -0.01999234],
       [-0.01770306,  0.34849166, -0.02047937]])

In [25]:
import torchgeometry as tgm
from torch.nn import functional as F

In [7]:
global_orient_seq = originalf['global_orient']
transl_seq = originalf['trans']

In [27]:
print(transf_rotmat.shape)

torch.Size([3, 3])


In [47]:
transf_rotmat.T.shape

torch.Size([3, 3])

In [53]:
tgm.angle_axis_to_rotation_matrix(global_ori_save).shape

torch.Size([10, 4, 4])

In [54]:
global_ori_save.shape

torch.Size([10, 3])

In [100]:
transf_transl = torch.Tensor(transf_transl)
delta_T  = torch.Tensor(delta_T)
transf_rotmat = torch.Tensor(transf_rotmat)

transf_rotate_xaxis = torch.Tensor(transf_rotate_xaxis)
delta_T_rotate = torch.Tensor(delta_T_rotate)
transf_transl_xaxis = torch.Tensor(transf_transl_xaxis) 
trans_save = torch.Tensor(testbp['transl'])
global_ori_save = torch.Tensor(testbp['global_orient'])

global_ori = tgm.angle_axis_to_rotation_matrix(global_ori_save)[:,:3,:3]
# global_ori_new = torch.einsum('ij,bjk->ik', transf_rotmat.permute(1,0), global_ori)
global_ori_new = torch.einsum('ij,tjk->tik', torch.linalg.inv(transf_rotmat.T),tgm.angle_axis_to_rotation_matrix(global_ori_save)[:,:3,:3])
f = F.pad(global_ori_new, [0,1])
f.shape
glo = tgm.rotation_matrix_to_angle_axis(f).contiguous()
# glorot = F.pad(global_ori_new, [0,1])
# glorot = tgm.rotation_matrix_to_angle_axis(glorot).view(-1, 3).contiguous()
transl = torch.matmul((torch.linalg.inv(transf_rotmat.T)),((trans_save[2:]+delta_T[2:]).T)).T+transf_transl-delta_T[2:]



In [99]:
glo.view(-1,3)

tensor([[1.3684, 0.9645, 0.9895],
        [1.3688, 0.9674, 0.9903],
        [1.3691, 0.9705, 0.9907],
        [1.3687, 0.9726, 0.9914],
        [1.3676, 0.9731, 0.9930],
        [1.3654, 0.9737, 0.9957],
        [1.3620, 0.9741, 1.0001],
        [1.3578, 0.9752, 1.0056],
        [1.3524, 0.9768, 1.0110],
        [1.3483, 0.9789, 1.0147]])

In [45]:
R.from_rotvec(global_ori_save).as_matrix().shape

(10, 3, 3)

In [70]:
testbp['transf_rotmat'] = data['transf_rotmat'] 
testbp['transf_transl'] = data['transf_transl'] 
testbp['delta_T'] = data['delta_T']
testbp['transf_rotate_xaxis'] = data['transf_rotate_xaxis']
testbp['transf_transl_xaxis'] = data['transf_transl_xaxis']
testbp['delta_T_rotate'] = data['delta_T_rotate']
transf_rotmat =testbp['transf_rotmat']
transf_transl =testbp['transf_transl']
delta_T=testbp['delta_T']
transf_rotate_xaxis=testbp['transf_rotate_xaxis']
transf_transl_xaxis=testbp['transf_transl_xaxis']
delta_T_rotate=testbp['delta_T_rotate']
transl_save = data['trans'][2:]
global_ori_save = data['global_orient'][2:]
transl_rotate_recover  = ((np.linalg.inv(transf_rotmat.T))@((transl_save+delta_T[2:]).T)).T+transf_transl-delta_T[2:]
global_ori_rotate_recover = R.from_matrix(np.einsum('ij,tjk->tik', np.linalg.inv(transf_rotmat.T),R.from_rotvec(global_ori_save).as_matrix())).as_rotvec()

transl_seq_recover = ((np.linalg.inv(transf_rotate_xaxis.T))@((transl_rotate_recover+delta_T_rotate[2:]).T)).T +transf_transl_xaxis-delta_T_rotate[2:]
global_orient_seq_recover = R.from_matrix(np.einsum('ij,tjk->tik', np.linalg.inv(transf_rotate_xaxis.T),R.from_rotvec(global_ori_rotate_recover).as_matrix())).as_rotvec()



In [71]:
global_ori_rotate_recover

array([[1.36913013, 0.97051295, 0.99074242],
       [1.36871214, 0.97259716, 0.99137432],
       [1.36756023, 0.97309061, 0.99304989],
       [1.36540973, 0.97370596, 0.99573563],
       [1.36203445, 0.97405217, 1.0001178 ],
       [1.35784646, 0.97516002, 1.00557762],
       [1.35243652, 0.97681134, 1.01101223],
       [1.34829781, 0.9788817 , 1.01474815]])

In [14]:
global_ori_rotate_recover

array([[1.36844411, 0.96448842, 0.98946256],
       [1.36881056, 0.96738951, 0.99030318],
       [1.36913018, 0.97051298, 0.99074241],
       [1.36871221, 0.97259721, 0.99137436],
       [1.36756025, 0.97309063, 0.99304983],
       [1.3654098 , 0.97370601, 0.99573551],
       [1.36203453, 0.97405223, 1.00011773],
       [1.35784643, 0.97516   , 1.00557764],
       [1.35243657, 0.97681137, 1.01101212],
       [1.34829786, 0.97888174, 1.01474806]])

In [17]:
global_ori_rotate = R.from_matrix(np.einsum('ij,tjk->tik', transf_rotate_xaxis.T, R.from_rotvec(global_orient_seq).as_matrix())).as_rotvec()
transl_rotate = np.einsum('ij,tj->ti', transf_rotate_xaxis.T, transl_seq+delta_T_rotate-transf_transl_xaxis)-delta_T_rotate

#calibrate
# transf_rotmat, transf_transl = get_new_coordinate(bodymodel_one, betas_seq[:1,:], transl_rotate[:1,:], pose_seq[:1,:],global_ori_rotate[:1,:])

# delta_T = calc_calibrate_offset(bodymodel_batch, betas_seq, transl_rotate, pose_seq,global_ori_rotate)

global_ori_save = R.from_matrix(np.einsum('ij,tjk->tik', transf_rotmat.T, R.from_rotvec(global_ori_rotate).as_matrix())).as_rotvec()
transl_save = np.einsum('ij,tj->ti', transf_rotmat.T, transl_rotate+delta_T-transf_transl)-delta_T


In [18]:
global_ori_rotate

array([[1.36844402, 0.96448832, 0.98946266],
       [1.36881053, 0.96738946, 0.99030311],
       [1.36913015, 0.97051292, 0.99074242],
       [1.36871216, 0.97259713, 0.99137432],
       [1.36756025, 0.97309059, 0.99304989],
       [1.36540975, 0.97370593, 0.99573563],
       [1.36203447, 0.97405215, 1.0001178 ],
       [1.35784647, 0.97516   , 1.00557762],
       [1.35243654, 0.97681131, 1.01101223],
       [1.34829783, 0.97888168, 1.01474815]])

In [177]:
transl_rotate_recover  = ((np.linalg.inv(transf_rotmat.T))@((transl_save+delta_T).T)).T+transf_transl-delta_T
global_ori_rotate_recover = R.from_matrix(np.einsum('ij,tjk->tik', np.linalg.inv(transf_rotmat.T),R.from_rotvec(global_ori_save).as_matrix())).as_rotvec()

transl_seq_recover = ((np.linalg.inv(transf_rotate_xaxis.T))@((transl_rotate_recover+delta_T_rotate).T)).T +transf_transl_xaxis-delta_T_rotate
global_orient_seq_recover = R.from_matrix(np.einsum('ij,tjk->tik', np.linalg.inv(transf_rotate_xaxis.T),R.from_rotvec(global_ori_rotate_recover).as_matrix())).as_rotvec()

In [178]:
global_orient_seq_recover

array([[-2.5225346 ,  0.02338093, -1.82928648],
       [-2.51955103,  0.02143599, -1.83144608],
       [-2.51672935,  0.01891195, -1.8335208 ],
       [-2.51495527,  0.01754975, -1.83559242],
       [-2.51399397,  0.01865696, -1.83785191],
       [-2.51275157,  0.02059953, -1.84158698],
       [-2.51111387,  0.02438859, -1.84715357],
       [-2.50852655,  0.02847916, -1.8545101 ],
       [-2.50620865,  0.0320554 , -1.86312297],
       [-2.50412559,  0.03364165, -1.86996499]])

In [179]:
original['global_orient']

array([[-2.52253461,  0.02338095, -1.82928646],
       [-2.51955104,  0.02143601, -1.83144605],
       [-2.51672935,  0.01891197, -1.83352077],
       [-2.51495528,  0.01754977, -1.83559239],
       [-2.51399398,  0.01865698, -1.83785188],
       [-2.51275158,  0.02059955, -1.84158695],
       [-2.51111388,  0.02438861, -1.84715354],
       [-2.50852656,  0.02847919, -1.85451007],
       [-2.50620866,  0.03205542, -1.86312294],
       [-2.5041256 ,  0.03364168, -1.86996496]])

In [166]:
global_ori_rotate

array([[1.36844402, 0.96448832, 0.98946266],
       [1.36881053, 0.96738946, 0.99030311],
       [1.36913015, 0.97051292, 0.99074242],
       [1.36871216, 0.97259713, 0.99137432],
       [1.36756025, 0.97309059, 0.99304989],
       [1.36540975, 0.97370593, 0.99573563],
       [1.36203447, 0.97405215, 1.0001178 ],
       [1.35784647, 0.97516   , 1.00557762],
       [1.35243654, 0.97681131, 1.01101223],
       [1.34829783, 0.97888168, 1.01474815]])

In [155]:
R.from_matrix(np.einsum('ij,tjk->tik', np.linalg.inv(transf_rotmat.T),R.from_rotvec(global_ori_save).as_matrix())).as_rotvec()

array([[1.368444  , 0.96448834, 0.98946266],
       [1.36881052, 0.96738948, 0.99030311],
       [1.36913013, 0.97051295, 0.99074242],
       [1.36871214, 0.97259716, 0.99137432],
       [1.36756023, 0.97309061, 0.99304989],
       [1.36540973, 0.97370596, 0.99573563],
       [1.36203445, 0.97405217, 1.0001178 ],
       [1.35784646, 0.97516002, 1.00557762],
       [1.35243652, 0.97681134, 1.01101223],
       [1.34829781, 0.9788817 , 1.01474815]])

In [153]:
transl_rotate_recover = ((np.linalg.inv(transf_rotmat.T))@((transl_save+delta_T).T)).T+transf_transl-delta_T

In [142]:
transl_seq = ((np.linalg.inv(transf_rotate_xaxis.T))@((transl_save+delta_T).T)).T+transf_transl-delta_T

array([[-6.77760367e-09,  8.88617871e-09,  0.00000000e+00],
       [ 2.18871659e-03, -2.20740512e-03, -5.94258308e-05],
       [ 4.49163298e-03, -4.48394533e-03, -1.65641308e-04],
       [ 6.53449196e-03, -6.63889300e-03, -3.77714634e-04],
       [ 8.35050052e-03, -8.32068219e-03, -7.55965710e-04],
       [ 9.97452566e-03, -9.89012766e-03, -1.22374296e-03],
       [ 1.09665707e-02, -1.09574795e-02, -1.92278624e-03],
       [ 1.20630072e-02, -1.19904882e-02, -2.82013416e-03],
       [ 1.28327838e-02, -1.27856856e-02, -3.45045328e-03],
       [ 1.39356028e-02, -1.35052790e-02, -3.93748283e-03]])

In [148]:
transl_rotate+delta_T-transf_transl

array([[ 0.00000000e+00, -1.11758709e-08,  0.00000000e+00],
       [-4.01616096e-04,  3.08250263e-03, -5.94258308e-05],
       [-8.52108002e-04,  6.28923252e-03, -1.65641308e-04],
       [-1.16956234e-03,  9.24156979e-03, -3.77714634e-04],
       [-1.59358978e-03,  1.16801150e-02, -7.55965710e-04],
       [-1.93309784e-03,  1.39129050e-02, -1.22374296e-03],
       [-2.07459927e-03,  1.53632052e-02, -1.92278624e-03],
       [-2.31993198e-03,  1.68495066e-02, -2.82013416e-03],
       [-2.44975090e-03,  1.79486163e-02, -3.45045328e-03],
       [-2.89022923e-03,  1.91895850e-02, -3.93748283e-03]])

In [94]:
a = transl_save + delta_T
a = np.einsum('ij,tj->ti',a,np.linalg.inv(transf_rotmat))

In [95]:
transl_rotate

array([[-0.00044799,  0.35737162, -0.01654189],
       [-0.0008496 ,  0.36045413, -0.01660131],
       [-0.0013001 ,  0.36366086, -0.01670753],
       [-0.00161755,  0.3666132 , -0.0169196 ],
       [-0.00204158,  0.36905174, -0.01729785],
       [-0.00238109,  0.37128453, -0.01776563],
       [-0.00252259,  0.37273483, -0.01846467],
       [-0.00276792,  0.37422113, -0.01936202],
       [-0.00289774,  0.37532024, -0.01999234],
       [-0.00333822,  0.37656121, -0.02047937]])

In [96]:
a

array([[-1.28150448e-17, -4.01616096e-04, -8.52108002e-04,
        -1.16956234e-03, -1.59358978e-03, -1.93309784e-03,
        -2.07459927e-03, -2.31993198e-03, -2.44975090e-03,
        -2.89022923e-03],
       [-1.11758709e-08,  3.08250263e-03,  6.28923252e-03,
         9.24156979e-03,  1.16801150e-02,  1.39129050e-02,
         1.53632052e-02,  1.68495066e-02,  1.79486163e-02,
         1.91895850e-02],
       [ 0.00000000e+00, -5.94258308e-05, -1.65641308e-04,
        -3.77714634e-04, -7.55965710e-04, -1.22374296e-03,
        -1.92278624e-03, -2.82013416e-03, -3.45045328e-03,
        -3.93748283e-03]])

In [26]:
transl_save

array([[-0.00044794,  0.35737169, -0.01654189],
       [-0.00323977,  0.35600461, -0.01660131],
       [-0.00613363,  0.35455147, -0.01670753],
       [-0.00882906,  0.35330578, -0.0169196 ],
       [-0.01100361,  0.35212358, -0.01729785],
       [-0.01301029,  0.3510873 , -0.01776563],
       [-0.01433901,  0.35048906, -0.01846467],
       [-0.0156686 ,  0.34978092, -0.01936202],
       [-0.01666835,  0.34930615, -0.01999234],
       [-0.01770306,  0.34849166, -0.02047937]])

In [29]:
inner(transl_rotate+delta_T-transf_transl, transf_rotmat.T)

NameError: name 'inner' is not defined

In [83]:
A = np.array([i for i in range (0,6,2)])
B = np.array([i for i in range (4,7)])

In [84]:
A = np.tile(A,(3,1))
B = np.tile(B,(3,1))

In [85]:
np.inner(A,B)

array([[34, 34, 34],
       [34, 34, 34],
       [34, 34, 34]])

In [76]:
np.einsum('ij,tj->ti',A,B)

array([[17, 17, 17],
       [17, 17, 17],
       [17, 17, 17]])

In [ ]:
for i in range (0,3):
    for j in range (0,

In [92]:
glorot = torch.Tensor(data['global_orient'])
transl = torch.Tensor(data['trans'])

In [88]:
transf_rotmat.repeat(3,1,1).shape

torch.Size([3, 3, 3])

In [ ]:
transf_rotmat = transf_rotmat

In [93]:
global_ori = tgm.angle_axis_to_rotation_matrix(glorot)[:,:3,:3]
global_ori_new = torch.einsum('bij,bjk->bik', transf_rotmat.T, global_ori)
glorot = F.pad(global_ori_new, [0,1])
glorot = tgm.rotation_matrix_to_angle_axis(glorot).view(-1, 3).contiguous()
transl = torch.einsum('bij,bj->bi', transf_rotmat.repeat(3,1,1).permute(0,2,1), transl+delta_T-transf_transl[:,0])-delta_T
        

RuntimeError: einsum(): the number of subscripts in the equation (3) does not match the number of dimensions (2) for operand 0 and no ellipsis was given